# Transpose Lower Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ y := L^T x + y $ where $ L $ is lower triangular.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## The `Tmvmult_lt_unb_var1!( L, x, y )` routine

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := L^T x + y $.  The "_lt_" in the name of the routine indicates this is the "lower triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Tmvmult_lt_unb_var1!(L, x, y)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(LTL, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.dots!( l21, x2, psi1 )
        laff.dots!( lambda11, chi1, psi1 )
        
        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Tmvmult_lt_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [11]:
L = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

# Notice that L is not lower triangular.  We will only use the lower triangular part.

println( "L before =" )
L

L before =


4×4 Array{Float64,2}:
 0.54334    0.996083   0.449281  0.865738
 0.0844183  0.990684   0.600359  0.381218
 0.390127   0.423369   0.277526  0.52265 
 0.292509   0.0274141  0.681395  0.799696

In [12]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.2909495194514651 
 0.21520796044117718
 0.1813166669223134 
 0.33717967063710574

In [13]:
println( "y before =" )
y

y before =


4-element Array{Float64,1}:
 0.10867568415850548
 0.49083173420982096
 0.47045948047131025
 0.5410419635726744 

In [14]:
laff.copy!(y, yold)
Tmvmult_lt_unb_var1!( L, x, y )

println( "y after = " )
y

y after = 


4-element Array{Float64,1}:
 0.4542921065640019
 0.7900420559148051
 0.7505320434771868
 0.8106832807103039

In [15]:
# LowerTriangular() makes the matrix lower triangular
using LinearAlgebra
println( "y - ( transpose( LowerTriangular( L ) ) * x + yold ) = " )
y - ( transpose( LowerTriangular( L ) ) * x + yold ) 

y - ( transpose( LowerTriangular( L ) ) * x + yold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## The `Tmvmult_lt_unb_var2( L, x, y )` routine

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := L^T x + y $.  The "_lt_" in the name of the routine indicates this is the "lower triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [16]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Tmvmult_lt_unb_var2!(L, x, y)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(LTL, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, lambda11, psi1 )
        laff.axpy!( chi1, l10t, y0 )
        
        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Tmvmult_lt_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [17]:
L = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

# Notice that L is not lower triangular.  We will only use the lower triangular part.

println( "L before =" )
L

L before =


4×4 Array{Float64,2}:
 0.0724212  0.446586  0.805223    0.307933
 0.71185    0.654529  0.0340174   0.832625
 0.426742   0.659015  0.00450574  0.791312
 0.182425   0.115015  0.636768    0.382166

In [18]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.8033201740435727
 0.9365308819590796
 0.3249564699055876
 0.7782984984742982

In [19]:
println( "y before =" )
y

y before =


4-element Array{Float64,1}:
 0.531017555273962 
 0.8805455307891643
 0.458780329045853 
 0.1842548005419482

In [20]:
laff.copy!(y, yold)
Tmvmult_lt_unb_var2!( L, x, y )

println( "y after = " )
y

y after = 


4-element Array{Float64,1}:
 1.536517707137722  
 1.7971992605074838 
 0.9558399521616561 
 0.48169428372477213

In [21]:
# LowerTriangular() makes the matrix lower triangular
using LinearAlgebra
println( "y - ( transpose( LowerTriangular( L ) ) * x + yold ) = " )
y - ( transpose( LowerTriangular( L ) ) * x + yold ) 

y - ( transpose( LowerTriangular( L ) ) * x + yold ) = 


4-element Array{Float64,1}:
 -2.220446049250313e-16
  2.220446049250313e-16
  0.0                  
  0.0                  

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.